# 실험해 볼 것
- fbprophet / ARIMA
- 윈도우 크기 조절
- '월' 변수 추가
- 결과를 여러번 내서 평균 내기
- +) 시가, 고가, 저가, 종가 적용해보기


In [16]:
# 드라이브 마운트하기
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#force_remount=True 는 기존 마운트와 상관없이 다시 마운트하라는 뜻

Mounted at /content/drive


In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout
from fbprophet import Prophet

In [18]:
train_df = pd.read_csv('/content/drive/MyDrive/농산물가격예측/public_data/train.csv')
train_df.head()

,date,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),얼갈이배추_거래량(kg),얼갈이배추_가격(원/kg),양배추_거래량(kg),양배추_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),당근_거래량(kg),당근_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),새송이_거래량(kg),새송이_가격(원/kg),팽이버섯_거래량(kg),팽이버섯_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg),청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,2016-01-01,금요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-02,토요일,80860.0,329.0,80272.0,360.0,122787.5,1281.0,3.0,11000.0,15019.0,5475.0,92334.0,1704.0,6359.0,1331.0,40028.0,348.0,4374.9,13242.0,16550.5,2339.0,10528.0,1729.0,13885.0,804.0,3853.0,3703.0,15797.0,2576.0,14634.0,1474.0,30950.0,1621.0,5125.0,9235.0,434.0,2109.0,19159.0,2414.0,880.0,2014.0,0.0,0.0
2,2016-01-03,일요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-01-04,월요일,1422742.5,478.0,1699653.7,382.0,2315079.0,1235.0,699.0,4464.0,141638.0,5210.0,994328.1,1716.0,262615.5,1212.0,1074699.1,345.0,122613.5,9923.0,427435.1,2153.0,82113.5,3960.0,558950.2,794.0,104930.3,4871.0,277326.5,2440.0,159800.0,1750.0,291057.0,1834.0,38525.5,7631.0,500702.0,2046.0,620539.0,2018.0,2703.8,3885.0,0.0,0.0
4,2016-01-05,화요일,1167241.0,442.0,1423482.3,422.0,2092960.1,1213.0,1112.6,4342.0,126207.8,5387.0,787716.0,1715.0,221850.5,1197.0,825681.9,350.0,79055.9,9529.0,334636.8,2220.0,80144.0,3333.0,444353.7,763.0,100699.5,5129.0,218465.2,2437.0,153084.0,1822.0,194626.5,1833.0,32615.0,6926.0,147638.0,2268.0,231958.0,2178.0,8810.0,2853.0,0.0,0.0


In [22]:
submission = pd.read_csv('/content/drive/MyDrive/농산물가격예측/sample_submission.csv')
date = submission[submission['예측대상일자'].str.contains('2020')]['예측대상일자'].str.split('+').str[0].unique()
date

array(['2020-09-29', '2020-09-30', '2020-10-01', '2020-10-02',
       '2020-10-03', '2020-10-04', '2020-10-05', '2020-10-06',
       '2020-10-07', '2020-10-08', '2020-10-09', '2020-10-10',
       '2020-10-11', '2020-10-12', '2020-10-13', '2020-10-14',
       '2020-10-15', '2020-10-16', '2020-10-17', '2020-10-18',
       '2020-10-19', '2020-10-20', '2020-10-21', '2020-10-22',
       '2020-10-23', '2020-10-24', '2020-10-25', '2020-10-26',
       '2020-10-27', '2020-10-28', '2020-10-29', '2020-10-30',
       '2020-10-31', '2020-11-01', '2020-11-02', '2020-11-03',
       '2020-11-04', '2020-11-05'], dtype=object)

In [19]:
# 평가식
def nmae(y_true, y_pred):
    score = np.mean(np.abs(y_true - y_pred) / y_true)
    return score

In [20]:
def make_dataset(data, column, date):
    test = pd.read_csv(f'/content/drive/MyDrive/농산물가격예측/public_data/test_files/test_{date}.csv')
    data = pd.concat([data, test])
    new = pd.DataFrame({'ds':data['date'], 'y':data[column]})
    return new

In [78]:
y_columns = list(train_df.columns[3::2])

for idx, d in enumerate(date):
    for col in y_columns:
        train = make_dataset(train_df, col, d)
        model = Prophet(daily_seasonality=True)
        model.fit(train)
        future = model.make_future_dataframe(periods=28)
        pred = model.predict(future)
        after_1w, after_2w, after_4w = pred['yhat'][-28:].iloc[[6, 13, 27]]
        submission.loc[[idx*3, idx*3+1, idx*3+2], col] = after_1w, after_2w, after_4w

In [79]:
submission.to_csv('/content/drive/MyDrive/농산물가격예측/fbprophet.csv', index=False)

In [80]:
submission

,예측대상일자,배추_가격(원/kg),무_가격(원/kg),양파_가격(원/kg),건고추_가격(원/kg),마늘_가격(원/kg),대파_가격(원/kg),얼갈이배추_가격(원/kg),양배추_가격(원/kg),깻잎_가격(원/kg),시금치_가격(원/kg),미나리_가격(원/kg),당근_가격(원/kg),파프리카_가격(원/kg),새송이_가격(원/kg),팽이버섯_가격(원/kg),토마토_가격(원/kg),청상추_가격(원/kg),백다다기_가격(원/kg),애호박_가격(원/kg),캠벨얼리_가격(원/kg),샤인마스캇_가격(원/kg)
0,2020-09-29+1week,1294.099913,783.177479,958.357483,17025.420046,4317.606127,1758.675132,1452.359875,982.223890,7164.869678,3277.028942,2779.888302,1648.257137,4354.462119,2294.591828,2068.396755,3524.741988,5346.113460,2110.568570,2080.751506,3705.447429,13902.645676
1,2020-09-29+2week,1115.751599,760.889910,959.960397,17241.965801,4414.888067,1671.049037,1438.532037,977.043770,6921.072190,3026.555399,2771.106461,1636.818028,4251.054622,2259.136924,2081.992672,3573.572480,5656.705799,2132.099093,2208.212000,3733.954652,13902.472543
2,2020-09-29+4week,939.955534,643.700299,1015.185148,17421.661765,4648.374294,1565.249273,1418.475010,951.883276,6820.197324,3112.482926,2710.932390,1662.757016,4611.906716,2266.113046,2074.412514,3322.799875,4460.609043,1991.204180,2553.336876,3925.421139,14672.806615
3,2020-09-30+1week,1265.181064,800.994216,949.972440,17076.751516,4350.847114,1742.929740,1426.549417,987.670914,7219.606276,3178.077491,2774.104503,1604.813055,4470.642605,2315.748591,2097.841411,3548.407862,5211.741963,2175.336833,2118.447410,3718.705120,13838.997070
4,2020-09-30+2week,1086.079119,769.615243,953.993169,17269.930915,4433.412033,1649.995862,1411.005598,976.890880,6985.125871,2950.333375,2731.521913,1595.667898,4417.381881,2277.594128,2109.301832,3590.989862,5475.765937,2183.026040,2271.821968,3752.122183,13835.346311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,2021-11-03+2week,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
224,2021-11-03+4week,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
225,2021-11-04+1week,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
226,2021-11-04+2week,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
